In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [3]:
import time
import copy
import itertools
from datetime import datetime,timedelta
from pathlib import Path

import csv 
import pickle

from os import listdir
from subprocess import Popen, PIPE


from scipy import stats
import numpy as np 

from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import minmax_scale

from tabulate import tabulate
import math


from labeling_utils import load_labels,stem_set
from fileUtils import read_file_properties

import pickle


In [4]:
INPUT_DIR_PARENT = "/home/data/nna/stinchcomb/NUI_DATA/"

mp3files_path="/home/enis/projects/nna/data/mp3files.txt"
mp3files,ignored=stem_set(mp3files_path)
file_properties,exceptions=read_file_properties(mp3files)

In [5]:

# tags=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Vehicle","Aircraft"]
tags=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Aircraft"]

tag_set=tags[:]


In [6]:
# LOAD LABELS by human
labelsbyhumanpath=Path('/scratch/enis/data/nna/labeling/results/')
# filter by username
labelsbyhuman=[i for i in listdir(labelsbyhumanpath) if (".csv" in i) ]

humanresults={}
counter=0
for apath in labelsbyhuman:
    with open(labelsbyhumanpath / apath, newline='') as f:
        reader=csv.reader(f)
        for row in reader:
            counter+=1
            humanresults[row[0]]=row[1:]

print("unique files:",len(humanresults),"\ntotal files",counter)

#Join vehicle and Aircraft
for file_name,tagshere in humanresults.items():
#     print(file_name,tagshere)
    
    tagshere=["Aircraft" if tag == "Vehicle" else tag for tag in tags]

unique files: 1300 
total files 1366


In [7]:
def vectorized_y_true(humanresults,tag_set):
    y_true={tag: [None]*len(humanresults) for tag in tag_set}
    for i,tags in enumerate(humanresults.values()):
        # we  only look for tags in tag_set
        for tag in tag_set:
            if tag in tags:
                y_true[tag][i] = 1
            else:
                y_true[tag][i] = 0 
    return y_true

y_true_dict=vectorized_y_true(humanresults,tag_set)


In [8]:
split_path=Path('/scratch/enis/data/nna/labeling/split_embeddings/')
# filter by username
split_embeds=[i for i in listdir(split_path) ]
raw_embeds =  [i for i in split_embeds if "rawembed" in i]
proc_embeds = [i for i in split_embeds if "_embed" in i]

In [14]:
h = .02  # step size in the mesh

names = [ 
         "Neural Net",]
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
short_names=[
 'NN',
]

# classifiers = [
#     MLPClassifier(alpha=1, max_iter=1000),]
classifiers = [
    MLPClassifier(max_iter=1000)]

In [15]:


def print_result(results):
    headers= ["Tag",]+short_names
    table=[]
#     sample_count=len(next(iter(y_true_dict.values())))
#     print("Total sample:",sample_count,"And threshold is",prob_threshold)
    for tag in (tag_set):
#         positive_count=sum(y_true_dict[tag])
        table.append([tag,*results[tag]])

    print(tabulate(table, headers=headers))

def reverse_results(results):
    results_clf2={}
    for tag,scores_per_Tag2 in results.items():
        scores_per_Tag=scores_per_Tag2[:]
        maximum=max(scores_per_Tag)
        scores_per_Tag=[color.BOLD + x + color.END if x==maximum else x for x in scores_per_Tag]

        for i,name in enumerate(names):
            score=scores_per_Tag[i]
            results_clf2.setdefault(name,[])
            results_clf2[name].append(score)      
    return results_clf2
        

def print_result2(results):
    results_clf=reverse_results(results)
    headers= ["Tag",]+list(tag_set)
    table=[]
#     sample_count=len(next(iter(y_true_dict.values())))
#     print("Total sample:",sample_count,"And threshold is",prob_threshold)
#     for tag in tag_set:
#         aresult=[]
    for i,name in enumerate(names):
#         positive_count=sum(y_true_dict[tag])
        table.append([short_names[i],*results_clf[name]])
    
    print(tabulate(table, headers=headers))


In [16]:
def get_results_save(X_input,y_true_dict,classifiers,names,tag_set,logs=False,many2one=False):
    results={}
    models={}

    
    for tag in (tag_set):
        data_start=time.time()
        X_train, X_test, y_train, y_test = train_test_split(
            X_input, y_true_dict[tag], test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.25, random_state=42)


        if many2one:
        # I modift y_train to be bigger to match X_train 
        # and I modify y_val_pred to be smaller to match y_val
#             if logs : print([len(i) for i in [X_train,X_test,X_val]])
            [X_train,X_test,X_val]=[np.reshape(x_array,(-1,128)) for x_array in [X_train,X_test,X_val]]

#             if logs : print([len(i) for i in [X_train,X_test,X_val]])

#             if logs : print([len(i) for i in [y_train,y_test,y_val]])

            for y_array in [y_train]:
                new_array=np.zeros((len(y_array)*10))
                for i,v in enumerate(y_array):
                    new_array[i*10:(i+1)*10]=v
                y_array[:]=new_array[:]
#             if logs : print([len(i) for i in [y_train,y_test,y_val]])
        data_end=time.time()
        if logs : print("data time",data_end-data_start)
        for name, clf in zip(names, classifiers):
            start=time.time()
            try:
#                 if logs : print(X_train.shape)
                clf.fit(X_train, y_train)
                models[name+"_"+tag]=(copy.deepcopy(clf))
                y_val_pred=clf.predict_proba(X_val)
#                 if logs :print(X_val.shape,y_val_pred.shape)

                if many2one:
                    excerpt_count=int(y_val_pred[:,1:].shape[0]/10)
                    if logs : print("excerpt_count",excerpt_count)
                    y_val_pred_new= np.empty((excerpt_count,2))
                    for i in range(0,y_val_pred.shape[0],10):
                        theslice=y_val_pred[i:i+10,1:]
                        value=np.max(theslice)
                        y_val_pred_new[int(i/10),1:]=value
                    y_val_pred=y_val_pred_new

                score=roc_auc_score(y_val,y_val_pred[:,1:])
    #             score = clf.score(X_val, y_val)
                score="{:.2}".format(score)

                results.setdefault(tag,[])

                results[tag].append(score)

                if logs : print(tag,name,score) 
            except:
                score="-1"

                results.setdefault(tag,[])

                results[tag].append(score)

                if logs : print(tag,name,score) 
            end=time.time()
            if logs : print("classifier time",end-start)
    return results,models

In [17]:
def map_reduce(X,y_true_dict,func_type):
    X_copy=X[:]
    y_true_dict_copy=copy.deepcopy(y_true_dict)
    if func_type=="Average":
        return np.mean(X_copy,axis=1),y_true_dict_copy
    elif func_type=="Concat":
        return np.reshape(X_copy,(-1,1280)),y_true_dict_copy
    elif func_type=="many2one":
#         for tag_name in y_true_dict_copy:
#             new_array=np.zeros((len(y_true_dict_copy[tag_name])*10))
#             for i,v in enumerate(y_true_dict_copy[tag_name]):
#                 new_array[i*10:(i+1)*10]=v
#             y_true_dict_copy[tag_name]=new_array[:]
        # will reverse this inside
        return np.reshape(X_copy,(-1,1280)),y_true_dict_copy
    else:
        raise Exception("ERROR with embed type")

def pick_embed(embed_type):
    # humanresults[proc_embeds[0].replace("_embed.npy",".mp3")]
    embedding_path= Path("/scratch/enis/data/nna/labeling/split_embeddings/")
    X=[]
    for i in humanresults:
        if embed_type=="Raw":
            file_name=i.replace(".mp3","_rawembed.npy")
        elif embed_type=="Normalized":
            file_name=i.replace(".mp3","_embed.npy")
        else:
            raise Exception("ERROR with embed type")
        an_x=np.load(embedding_path / file_name)
        X.append(an_x)
    return np.array(X)

In [18]:
# params
embed_types=["Raw"] # "Raw" or "Normalized"
map_reduce_embeds= ["many2one"]

OUTPUT_DIR = "/scratch/enis/data/nna/NUI_DATA/"
OUTPUT_DIR = Path(OUTPUT_DIR)


In [19]:

all_results={}
all_models={}
for embed_type in embed_types:
    for map_reduce_type in map_reduce_embeds:
        X=pick_embed(embed_type)
        X,y_dict=map_reduce(X,y_true_dict,map_reduce_type)
        print("\n {} embeddings and merged by {} ".format(embed_type,map_reduce_type))
        many2one= True if map_reduce_type=="many2one" else False
        results,models=get_results_save(X,y_dict,classifiers,names,tag_set,logs=False,many2one=many2one)
        all_results[embed_type+"_"+map_reduce_type]=results
        all_models[embed_type+"_"+map_reduce_type]=models
        print_result2(results)



 Raw embeddings and merged by many2one 
Tag      Songbird    Water Bird    Insect    Running Water    Rain    Cable    Wind    Aircraft
-----  ----------  ------------  --------  ---------------  ------  -------  ------  ----------
NN           0.87          0.75      0.75              0.9    0.81     0.89    0.87        0.47


In [20]:

from datetime import datetime
timestr=datetime.now().strftime("%Y-%m-%d--%H:%M")

with open('raw_many2one_NN_'+timestr+'.pkl', 'wb') as f:
    pickle.dump(all_models, f)

In [7]:
with open('raw_many2one_NN_12_04_Y-00:18.pkl', 'rb') as f:
    all_models=pickle.load(f)

In [20]:
all_models['Raw_many2one'].keys()
# all_models.keys()

dict_keys(['Neural Net_Songbird', 'Neural Net_Water Bird', 'Neural Net_Insect', 'Neural Net_Running Water', 'Neural Net_Rain', 'Neural Net_Cable', 'Neural Net_Wind', 'Neural Net_Aircraft', 'Raw_many2one'])

In [15]:
def many2one_predict(X,clf):
    res=clf.predict(X)
    resSpilled=res[-(res.size%10):]
    resRest=res[:-(res.size%10)]
    resSpilled=np.max(resSpilled)
    resRest=resRest.reshape(-1,10)
    resRest=np.max(resRest,axis=1)
    resRest=np.append(resRest,[resSpilled])
    return resRest

In [17]:
#TEST
samples=np.ones((200,128))
many2one_predict(samples,all_models['Raw_many2one']['Neural Net_Songbird'])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [28]:
def find_raw(searchpath):
    command_list=['find', str(searchpath), '-iname', '*rawembeddings*',]
    process = Popen(command_list, stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()

    if process.returncode!=0:
        print('Error: '  + stderr.decode())
    files=stdout.decode()
    files=sorted(files.split("\n"))
    files=list(filter(lambda x: len(x)>0,files))
    return files

In [ ]:
from pathlib import Path
searchpath= 
searchpath=Path(searchpath)
files=

In [30]:
selected_areas=['CLVL5',
'ITKILLIK2',
'CLVL4',
'UMIAT',
'OCNPT',
'CLVL2',
'ITKILLIK1',
'USGS']

selected_areas_dict= { i:[] for i in selected_areas}

for key,f in file_properties.items():
    if f["site_id"] in selected_areas:
        selected_areas_dict[f["site_id"]].append(key)
#         print(f)
        

In [31]:
# # READ FILE NAMES, and RECORD timedelta,    NO  total 
# for files49 in selected_areas_dict.values():
#     for file in files49:
# #         print(file)
#         x=file_properties[file]
# #         print(x)
#         hour_min_sec=x["hour_min_sec"]
#         hour=int(hour_min_sec[:2])
#         minute=int(hour_min_sec[2:4])
#         second=int(hour_min_sec[4:6])
#         year = int(x["year"])
#         if year==2013:
#             year=2016
#         mm=datetime(year, int(x["month"]), int(x["day"]), hour=hour, minute=minute, second=second, microsecond=0)
#         x["timestamp"]=mm
# #         x["timedelta"] = timedelta(seconds=total[Path(mp3file)])


# IMPORTANT, we use order as timeline
# SORT BY TIME SO THAT INFERENCE FILES ARE ORDERED AS WELL
for i,area in enumerate(selected_areas_dict):
#     print(mp3files_unsorted)
    selected_areas_dict[area]=list(set(selected_areas_dict[area]))
    selected_areas_dict[area].sort( key=lambda x: file_properties[Path(x)]["timestamp"])   


In [18]:
OUTPUT_DIR / mp3file.relative_to(INPUT_DIR_PARENT).parent / (mp3file.stem +"_vgg")

NameError: name 'OUTPUT_DIR' is not defined

In [35]:
model_tag_names=["Neural Net_Songbird",
"Neural Net_Water Bird",
"Neural Net_Insect",
"Neural Net_Running Water",
"Neural Net_Rain",
"Neural Net_Cable",
"Neural Net_Wind",
"Neural Net_Aircraft",]

In [38]:
gathered_results_perTag={}


for i,longmp3files in enumerate(selected_areas_dict.values()):   
    for mp3file in longmp3files:
        vgg_folder= OUTPUT_DIR / mp3file.relative_to(INPUT_DIR_PARENT).parent / (mp3file.stem +"_vgg")  

        embedding_files=find_raw(vgg_folder)

        for embeddingfile in (embedding_files):
            embeddings=np.load(embeddingfile)
        # ['Neural Net_Songbird','Neural Net_Insect','Neural Net_Aircraft']
            for m_name in model_tag_names:
                the_model=all_models['Raw_many2one'][m_name]

#                 gathered = predict_reduce(embeddings,the_model,sample_per_tick=360)
                mapped = many2one_predict(embeddings,the_model)
                
                results= mapped
                gathered_results_perTag.setdefault(m_name,{})
                gathered_results = gathered_results_perTag[m_name]
                
                gathered_results.setdefault(mp3file,np.empty((0)))
                
                gathered_results[mp3file]= np.concatenate((gathered_results[mp3file],results))
        print(mp3file)

    print(i,longmp3files)
       

/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160602_163306.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160604_181421.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160606_195535.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160608_213649.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160610_231803.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160613_005919.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160615_024034.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160617_042148.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160619_060304.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160621_074419.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/June 2016/CLVL5_20160623_092534.MP3
/home/data/nna/stinchcomb/NUI_DATA/05 Colville 5/July 2016/CLVL5_

/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160602_154908.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160604_173026.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160606_191142.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160608_205258.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160610_223415.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160613_001533.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160615_015650.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160617_033807.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160619_051925.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160621_070042.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160623_084159.MP3
/home/data/nna/stinchcomb/NUI_DATA/04 Colville 4/July 2016/CLVL4_

/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160624_200541.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160626_214658.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160628_232813.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160701_010928.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160703_025043.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160705_043158.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160707_061314.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160709_075429.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160711_093545.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160713_111701.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/July 2016/OCNPT_20160715_125817.MP3
/home/data/nna/stinchcomb/NUI_DATA/03 Ocean Pt/August 2016/OCNPT_20160727_151103.MP3
/h

/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160623_194501.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160625_212615.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160627_230728.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160630_004840.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160702_022953.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160704_041106.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160706_055218.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160708_073331.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160710_091444.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160712_105557.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkillik/July 2016/ITKILLIK1_20160714_123710.MP3
/home/data/nna/stinchcomb/NUI_DATA/01 Itkil

/home/data/nna/stinchcomb/NUI_DATA/09 USGS/PassOver2_USGS_20160610_162930.mp3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/PassOver_USGS_20160610_160246.mp3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/July 2016/USGS_20160719_083150.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/(3)TURBOPROPorLIGHTWEIGHT_USGS_20160610_161304.mp3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0721_144426.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0723_162541.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0725_180656.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0727_194812.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0729_212927.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0731_231042.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0803_005157.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0805_023311.MP3
/home/data/nna/stinchcomb/NUI_DATA/09 USGS/August 2016/0807_041426.MP3
/home/data/nna/stinchcomb/NUI_DATA

In [52]:
np.save("8tags_on_8sites.npy",gathered_results_perTag)

In [ ]:
# np.save("inference_on_8sites.npy",gathered_results)

In [ ]:
# THIS PART JUST FILLS TOTAL LENGTH
gathered_results={}
# histogram={}
total={}


for i,mp3file in enumerate(mp3files):
    mp3file=Path(mp3file)
    total.setdefault(mp3file,0)
    
    vgg_folder= OUTPUT_DIR / mp3file.relative_to(INPUT_DIR_PARENT).parent / (mp3file.stem +"_vgg")  

    embedding_files=find_raw(vgg_folder)

    for embeddingfile in (embedding_files):
        embeddings=np.load(embeddingfile)
        seconds=embeddings.shape[0]
        total[mp3file]+=seconds
    print(i,total[mp3file])


            

In [ ]:
#  TEST, print start, end, total_days

all_tuples=[]
for i,area in enumerate(selected_areas_dict):
#     print(selected_areas_dict[area])
#     print(file_properties[Path(m)]["timestamp"])
    
    start=selected_areas_dict[area][0]
    end=selected_areas_dict[area][-1]
    print(start)
    print("start",file_properties[Path(start)]["timestamp"])
    print("end",file_properties[Path(end)]["timestamp"])
    days=(histogram[file_properties[Path(start)]["site_id"]]/(3600*24))
    all_tuples.append((file_properties[Path(start)]["timestamp"],file_properties[Path(end)]["timestamp"],start,days))

all_tuples.sort()
                                    
for i in all_tuples:
    print(i[2])
    print(i[0],i[1],i[3])